In [1]:
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import math

path_to_jsonfiles = "./rawData"
candidates = []
for file in os.listdir(path_to_jsonfiles):
    counter_id = 1
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        counter = 1
        for cand in dict["recommendations"]["items"]:
            candidate = {}
            candidate["id"] = counter
            counter_id+=1
            candidate["name"] = "name" 
            candidate["score"] = cand["rank_score"]
            candidate["tag1"]= cand["rabbit_rating"]
            candidate["tag2"]= cand["reliability_rate"]
            candidate["tag3"]= cand["category_invoices_count"]
            candidates.append(candidate)
            counter +=1
            if counter == 11:
                break
genders = ['m','m','m','f','m','m','f','m','m','f']
counter = 0
totaldata = {}
for cand in candidates:
    cand["gender"]=genders[counter]
    counter+=1
with open('./cleanedData/help_moving.json', 'w') as outfile:
    totaldata["totaldata"] = candidates
    json.dump(totaldata, outfile, indent=4)

In [2]:
def linkedInFair(candidates, maleRatio, femaleRatio):
    female_candidates = []
    male_candidates = []
    for cand in candidates:
            if cand["gender"] == "f":
                female_candidates.append(cand)
            else:
                male_candidates.append(cand)
    female_candidates = sorted(female_candidates, key=lambda k: k['score'], reverse=True)
    male_candidates = sorted(male_candidates, key=lambda k: k['score'], reverse=True) 
    protectedRanked = 0
    protected_2_ranked = 0
    ranking = []
    for k in range(1,len(candidates)+1):
        minProtectedrequired = round(k*femaleRatio)#0.42
        minProtected_2_required = round(k*maleRatio)#0.58
        if protectedRanked< minProtectedrequired and len(female_candidates)>=1 and minProtected_2_required <= protected_2_ranked:
            cand = female_candidates[0]
            ranking.append(cand)
            protectedRanked+=1
            candidates.remove(cand)
            female_candidates.remove(cand)
        elif protectedRanked< minProtectedrequired and len(female_candidates)>=1 and minProtected_2_required > protected_2_ranked:
            candidate = candidates[0]
            if candidate["gender"] == "f":
                protectedRanked+=1
                female_candidates.remove(candidate)
                candidates.remove(candidate)
                ranking.append(candidate)
            else:
                male_candidates.remove(candidate)
                candidates.remove(candidate)
                protected_2_ranked+=1
                ranking.append(candidate)
        elif protectedRanked>= minProtectedrequired and len(male_candidates)>=1 and minProtected_2_required > protected_2_ranked:
            candidate = male_candidates[0]
            male_candidates.remove(candidate)
            candidates.remove(candidate)
            protected_2_ranked+=1
            ranking.append(candidate)
        else:
            candidate = candidates[0]
            if candidate["gender"] == "f":
                protectedRanked+=1
                female_candidates.remove(candidate)
                candidates.remove(candidate)
                ranking.append(candidate)
            else:
                male_candidates.remove(candidate)
                candidates.remove(candidate)
                protected_2_ranked+=1
                ranking.append(candidate)
    for i in range(0,len(ranking)):
        cand = ranking[i]
        cand["fairrank"] = i+1
    ranking = sorted(ranking, key=lambda k: k['score'], reverse=True)
    for i in range(0,len(ranking)):
        cand = ranking[i]
        cand["scorerank"] = i+1
    ranking = sorted(ranking, key=lambda k: k['fairrank'])
    return ranking

In [10]:
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import math

path_to_jsonfiles = "./cleanedData"
candidates = []
for file in os.listdir(path_to_jsonfiles):
    counter_id = 1
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        female_candidates = []
        male_candidates = []
        candidates_old = dict["totaldata"]
        candidates_old = sorted(candidates_old, key=lambda k: k['score'], reverse=True)
        ranking = sorted(candidates_old, key=lambda k: k['fairrank'])
        exposure_women = 0
        exposure_men = 0
        rank = 1
        stringlist_new = []
        stringlist_old = []
        dcg=0
        for cand in ranking:
            stringlist_new.append(""+str(cand["gender"]) + " ; "+str(cand["score"]))
            if cand["gender"] == "f":
                exposure_women+=(1/math.log(rank+1,2))
            else:
                exposure_men +=(1/math.log(rank+1,2))
            rank+=1
            dcg+=(cand["score"]/math.log(rank+1,2))
        stringlist_new.append("Avg exposure f: "+str(exposure_women/3))
        stringlist_new.append("Avg exposure m: "+str(exposure_men/7))
        stringlist_new.append("DCG@10: "+str(dcg))
        exposure_women = 0
        exposure_men = 0
        dcg = 0
        rank = 1
        for cand in candidates_old:
            if cand["gender"] == "f":
                exposure_women+=(1/math.log(rank+1,2))
            else:
                exposure_men +=(1/math.log(rank+1,2))
            dcg+=(cand["score"]/math.log(rank+1,2))
            rank+=1
            stringlist_old.append(""+str(cand["gender"]) + " ; "+str(cand["score"])+'\t'+'\t'+'\t')
        stringlist_old.append("Avg exposure f: "+str(exposure_women/3)+'\t')
        stringlist_old.append("Avg exposure m: "+str(exposure_men/7)+'\t')
        stringlist_old.append("DCG@10: "+str(dcg)+'\t'+'\t')
        print(file)
        print("-----rabbit ranking"+'\t'+'\t'+'\t'+"|"+'\t'+"fair ranking-----")
        for i in range(0,len(stringlist_new)):
            print(stringlist_old[i] +"|"+stringlist_new[i])
        
        #json.dump(totaldata, outfile, indent=4)

    

event_staffing.json
-----rabbit ranking			|	fair ranking-----
m ; 0.799641978367866			|m ; 0.799641978367866
m ; 0.788158103114649			|f ; 0.7412400221038719
m ; 0.7805555452827792			|m ; 0.788158103114649
m ; 0.7710606362667689			|f ; 0.7258889517568552
m ; 0.742341028683199			|m ; 0.7805555452827792
f ; 0.7412400221038719			|f ; 0.7030471958586693
m ; 0.7375636107859171			|m ; 0.7710606362667689
f ; 0.7258889517568552			|m ; 0.742341028683199
m ; 0.719530008545884			|m ; 0.7375636107859171
f ; 0.7030471958586693			|m ; 0.719530008545884
Avg exposure f: 0.32024563007054624	|Avg exposure f: 0.47260449958429085
Avg exposure m: 0.5118317782681009	|Avg exposure m: 0.44653511990506756
DCG@10: 3.4651549603645355		|DCG@10: 2.888400868035004
event_staffing_swapped.json
-----rabbit ranking			|	fair ranking-----
f ; 0.799641978367866			|f ; 0.799641978367866
f ; 0.788158103114649			|m ; 0.7412400221038719
f ; 0.7805555452827792			|f ; 0.788158103114649
f ; 0.7710606362667689			|m ; 0.72588895175

In [3]:
path_to_jsonfiles = "./cleanedData"
candidates = []
for file in os.listdir(path_to_jsonfiles):
    counter_id = 1
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        totaldata = {}
        candidates = []
        for cand in dict["totaldata"]:
            if cand["gender"] == 'f':
                cand["gender"] = 'm'
            else:
                cand["gender"] = 'f'
            candidates.append(cand)
        totaldata["totaldata"] = candidates
        with open('./cleanedData/'+file[:len(file)-5]+'_swapped.json', 'w') as outfile:
            json.dump(totaldata, outfile, indent=4)

In [4]:
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import math
path_to_jsonfiles = "./cleanedData"
candidates = []
for file in os.listdir(path_to_jsonfiles):
    counter_id = 1
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        totaldata = {}
        if "swapped" in file:
            candidates = linkedInFair(dict["totaldata"], 0.42,0.58)
        else:
            candidates = linkedInFair(dict["totaldata"], 0.58,0.42)
        totaldata["totaldata"] = candidates
        with open('./cleanedData/'+file, 'w') as outfile:
            json.dump(totaldata, outfile, indent=4)

In [11]:
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import math
path_to_jsonfiles = "./cleanedData"
counter_id = 1
for file in os.listdir(path_to_jsonfiles):
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        totaldata = {}
        candidates = []
        for cand in dict["totaldata"]:
            cand["id"] = counter_id
            candidates.append(cand)
            counter_id+=1
        totaldata["totaldata"] = candidates
        with open('./cleanedData/'+file, 'w') as outfile:
            json.dump(totaldata, outfile, indent=4)

In [23]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import os, json
import pandas as pd
import numpy as np
import seaborn as sns
import math
import statistics

path_to_jsonfiles = "./cleanedData"
candidates = []
for file in os.listdir(path_to_jsonfiles):
    counter_id = 1
    full_filename = "%s/%s" % (path_to_jsonfiles, file)
    with open(full_filename,'r') as fi:
        if 'swapped' in file:
            continue
        dict = json.load(fi)
        f_score = []
        m_score = []
        f_tasks = []
        m_tasks = []
        f_reliability = []
        m_reliability = []
        f_positive = []
        m_positive = []
        for cand in dict["totaldata"]:
            if cand["gender"] == 'f':
                f_score.append(cand["score"])
                f_tasks.append(cand["tag3"])
                f_reliability.append(cand["tag2"])
                f_positive.append(int(cand["tag1"][:len(cand["tag1"])-1]))
            else:
                m_score.append(cand["score"])
                m_tasks.append(cand["tag3"])
                m_reliability.append(cand["tag2"])
                m_positive.append(int(cand["tag1"][:len(cand["tag1"])-1]))
        print("TASKS")
        print("MALE: "+file+"-: "+str((round(sum(m_tasks)/len(m_tasks),2))) + " sd " + str(round(statistics.stdev(m_tasks),2)))
        print("FEMALE: "+file+"-: "+str((round(sum(f_tasks)/len(f_tasks),2))) + " sd " + str(round(statistics.stdev(f_tasks),2))+'\n')
        print("REVIEWS")
        print("MALE: "+file+"-: "+str((round(sum(m_positive)/len(m_positive),2))) + " sd " + str(round(statistics.stdev(m_positive),2)))
        print("FEMALE: "+file+"-: "+str((round(sum(f_positive)/len(f_positive),2))) + " sd " + str(round(statistics.stdev(f_positive),2))+'\n')
        print("RELIABILITY")
        print("MALE: "+file+"-: "+str((round(sum(m_reliability)/len(m_reliability),2))) + " sd " + str(round(statistics.stdev(m_reliability),2)))
        print("FEMALE: "+file+"-: "+str((round(sum(f_reliability)/len(f_reliability),2))) + " sd " + str(round(statistics.stdev(f_reliability),2))+'\n')
        print("SCORE")
        print("MALE: "+file+"-: "+str((round(sum(m_score)/len(m_score),2))) + " sd " + str(round(statistics.stdev(m_score),2)))
        print("FEMALE: "+file+"-: "+str((round(sum(f_score)/len(f_score),2))) + " sd " + str(round(statistics.stdev(f_score),2))+'\n')
        print("----------------------------------------------------------------------------------")

TASKS
MALE: event_staffing.json-: 1.57 sd 2.15
FEMALE: event_staffing.json-: 5.0 sd 4.36

REVIEWS
MALE: event_staffing.json-: 98.14 sd 1.77
FEMALE: event_staffing.json-: 97.0 sd 3.61

RELIABILITY
MALE: event_staffing.json-: 96.29 sd 8.2
FEMALE: event_staffing.json-: 100.0 sd 0.0

SCORE
MALE: event_staffing.json-: 0.76 sd 0.03
FEMALE: event_staffing.json-: 0.72 sd 0.02

----------------------------------------------------------------------------------
TASKS
MALE: help_moving.json-: 306.43 sd 220.74
FEMALE: help_moving.json-: 63.0 sd 69.66

REVIEWS
MALE: help_moving.json-: 99.14 sd 1.21
FEMALE: help_moving.json-: 99.67 sd 0.58

RELIABILITY
MALE: help_moving.json-: 99.71 sd 0.76
FEMALE: help_moving.json-: 100.0 sd 0.0

SCORE
MALE: help_moving.json-: 0.85 sd 0.01
FEMALE: help_moving.json-: 0.84 sd 0.01

----------------------------------------------------------------------------------
TASKS
MALE: shopping.json-: 0.57 sd 0.79
FEMALE: shopping.json-: 0.0 sd 0.0

REVIEWS
MALE: shopping.json-: